In [2]:
# coding: utf-8
# ライブラリのインポート
import os
import sys

import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

In [22]:
# trainとtestデータを用意 x_train(200,4)t_train(200,2)x_test(52,4)t_test(52,2)

os.chdir("C:\\Users\\TakaiYohei\\Desktop\\illust_evaluator")

t_data_f = pd.read_csv('illust_dataset_eval.csv')

print("Before", t_data_f.shape)

t_data_f = t_data_f.drop(['illust_number'], axis=1)

"After", t_data_f.shape

Before (252, 4)


('After', (252, 3))

In [23]:
t_data_f.head()

,color,multi,score
0,1,0,94.318572
1,1,0,21.864456
2,1,0,56.975131
3,1,0,72.123115
4,1,0,99.712607


In [25]:
# trainとtestデータを用意 x_train(200,4)t_train(200,2)x_test(52,4)t_test(52,2)

os.chdir("C:\\Users\\TakaiYohei\\Desktop\\illust_evaluator")

t_data_f = pd.read_csv('illust_dataset_eval.csv')

t_data = t_data_f.as_matrix()

train_matrix, test_matrix = np.split(t_data, [200])

x_train = []
t_train_s = []
for x in train_matrix:
    x_split = np.hsplit(x, [2])
    x_train.append(x_split[0].astype(np.float32))
    t_train_s.append(x_split[1].astype(np.int32))

x_train = np.array(x_train)
t_train_s = np.array(t_train_s)

x_test = []
t_test_s = []
for x in test_matrix:
    x_split = np.hsplit(x, [2])
    x_test.append(x_split[0].astype(np.float32))
    t_test_s.append(x_split[1].astype(np.int32))
    
x_test = np.array(x_test)
t_test_s = np.array(t_test_s)

t_train = t_train_s
t_test = t_test_s

t_train = np.array(t_train)
t_test = np.array(t_test)

In [4]:
# レイヤの定義
os.chdir("C:\\Users\\TakaiYohei\\Desktop\\deep-learning-from-scratch-master\\ch05")

import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads


In [5]:
# mnistをロード
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
#(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)


In [6]:

network = TwoLayerNet(input_size=2, hidden_size=2, output_size=1)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 20
learning_rate = 0.000001

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

ValueError: shapes (20,4) and (6,15) not aligned: 4 (dim 1) != 6 (dim 0)

In [6]:
A = np.array([[238], [9], [7], [324], [1], [0]]).astype(np.float32).transpose()
network.predict(A)

array([[-0.0664802,  0.0031686]])

In [7]:
B = np.array([[110], [10], [4], [40], [4], [0]]).astype(np.float32).transpose()
network.predict(B)

array([[-0.09309014,  0.01905264]])

In [8]:
C = np.array([[359], [8], [0], [324], [1], [0]]).astype(np.float32).transpose()
network.predict(C)

array([[-0.18947837,  0.03058765]])